### Qdrant, datasets

In memory vector search

In [3]:
import rich
from datasets import Dataset, load_dataset
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models

In [4]:
issues_dataset = load_dataset("lewtun/github-issues", split="train")

Repo card metadata block was not found. Setting CardData to empty.


In [5]:
issues_dataset = issues_dataset.filter(
    lambda x: (x["is_pull_request"] is False and len(x["comments"]) > 0)
)

columns = issues_dataset.column_names
columns_to_keep = ["title", "body", "html_url", "comments"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
issues_dataset = issues_dataset.remove_columns(columns_to_remove)


In [6]:
issues_dataset.set_format("pandas")
df = issues_dataset[:]
# Split each comment into an individual record
comments_df = df.explode("comments", ignore_index=True)

In [7]:
comments_dataset = Dataset.from_pandas(comments_df)

comments_dataset = comments_dataset.map(
    lambda x: {"comment_length": len(x["comments"].split())}
)
# Filter short comments
comments_dataset = comments_dataset.filter(lambda x: x["comment_length"] > 15)

Filter: 100%|██████████| 2964/2964 [00:00<00:00, 264689.09 examples/s]


In [8]:
def concatenate_text(examples):
    return {
        "text": examples["title"]
        + " \n "
        + examples["body"]
        + " \n "
        + examples["comments"]
    }


comments_dataset = comments_dataset.map(concatenate_text)

Map: 100%|██████████| 2175/2175 [00:00<00:00, 22389.69 examples/s]


In [9]:
client = QdrantClient(":memory:")
embedding_model = TextEmbedding()

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 33770.56it/s]


In [10]:
client.create_collection(
    collection_name="github_issues",
    vectors_config=models.VectorParams(
        size=384,  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

True

In [11]:
client.upload_points(
    collection_name="github_issues",
    points=[
        models.PointStruct(id=idx, vector=list(embedding_model.embed(doc["text"]))[0], payload=doc)
        for idx, doc in enumerate(comments_dataset)
    ],
)

In [12]:
hits = client.search(
    collection_name="github_issues",
    query_vector=list(embedding_model.embed("How can I load a dataset offline?"))[0],
    limit=3,
)
for hit in hits:
    rich.print(hit.payload, "score:", hit.score)

{
    'html_url': 'https://github.com/huggingface/datasets/issues/824',
    'title': 'Discussion using datasets in offline mode',
    'comments': 'I opened a PR that allows to reload modules that have already been loaded once even if there\'s no
internet.\r\n\r\nLet me know if you know other ways that can make the offline mode experience better. I\'d be happy
to add them :) \r\n\r\nI already note the "freeze" modules option, to prevent local modules updates. It would be a 
cool feature.\r\n\r\n----------\r\n\r\n> @mandubian\'s second bullet point suggests that there\'s a workaround 
allowing you to use your offline (custom?) dataset with `datasets`. Could you please elaborate on how that should 
look like?\r\n\r\nIndeed `load_dataset` allows to load remote dataset script (squad, glue, etc.) but also you own 
local ones.\r\nFor example if you have a dataset script at `./my_dataset/my_dataset.py` then you can 
do\r\n```python\r\nload_dataset("./my_dataset")\r\n```\r\nand the dataset script will generate your dataset once 
and for all.\r\n\r\n----------\r\n\r\nAbout I\'m looking into having `csv`, `json`, `text`, `pandas` dataset 
builders already included in the `datasets` package, so that they are available offline by default, as opposed to 
the other datasets that require the script to be downloaded.\r\ncf #1724 ',
    'body': '`datasets.load_dataset("csv", ...)` breaks if you have no connection (There is already this issue 
https://github.com/huggingface/datasets/issues/761 about it). It seems to be the same for metrics too.\r\n\r\nI 
create this ticket to discuss a bit and gather what you have in mind or other propositions.\r\n\r\nHere are some 
points to open discussion:\r\n- if you want to prepare your code/datasets on your machine (having internet 
connexion) but run it on another offline machine (not having internet connexion), it won\'t work as is, even if you
have all files locally on this machine.\r\n- AFAIK, you can make it work if you manually put the python files 
(csv.py for example) on this offline machine and change your code to `datasets.load_dataset("MY_PATH/csv.py", 
...)`. But it would be much better if you could run ths same code without modification if files are available 
locally.\r\n- I\'ve also been considering the requirement of downloading Python code and execute on your machine to
use datasets. This can be an issue in a professional context. Downloading a CSV/H5 file is acceptable, downloading 
an executable script can open many security issues. We certainly need a mechanism to at least "freeze" the dataset 
code you retrieved once so that you can review it if you want and then be sure you use this one everywhere and not 
a version dowloaded from internet.\r\n \r\nWDYT? (thks)\r\n\r\n',
    'comment_length': 179,
    'text': 'Discussion using datasets in offline mode \n `datasets.load_dataset("csv", ...)` breaks if you have no
connection (There is already this issue https://github.com/huggingface/datasets/issues/761 about it). It seems to 
be the same for metrics too.\r\n\r\nI create this ticket to discuss a bit and gather what you have in mind or other
propositions.\r\n\r\nHere are some points to open discussion:\r\n- if you want to prepare your code/datasets on 
your machine (having internet connexion) but run it on another offline machine (not having internet connexion), it 
won\'t work as is, even if you have all files locally on this machine.\r\n- AFAIK, you can make it work if you 
manually put the python files (csv.py for example) on this offline machine and change your code to 
`datasets.load_dataset("MY_PATH/csv.py", ...)`. But it would be much better if you could run ths same code without 
modification if files are available locally.\r\n- I\'ve also been considering the requirement of downloading Python
code and execute on your machine to use datasets. This can be an issue in a professional context. Downloading a 
CSV/H5 file is acceptable, downloading an executable script can open many sec

{
    'html_url': 'https://github.com/huggingface/datasets/issues/824',
    'title': 'Discussion using datasets in offline mode',
    'comments': 'here is my way to load a dataset offline, but it **requires** an online machine\r\n1. (online 
machine)\r\n```\r\nimport datasets\r\ndata = 
datasets.load_dataset(...)\r\ndata.save_to_disk(/YOUR/DATASET/DIR)\r\n```\r\n2. copy the dir from online to the 
offline machine\r\n3. (offline machine)\r\n```\r\nimport datasets\r\ndata = 
datasets.load_from_disk(/SAVED/DATA/DIR)\r\n```\r\n\r\nHTH.',
    'body': '`datasets.load_dataset("csv", ...)` breaks if you have no connection (There is already this issue 
https://github.com/huggingface/datasets/issues/761 about it). It seems to be the same for metrics too.\r\n\r\nI 
create this ticket to discuss a bit and gather what you have in mind or other propositions.\r\n\r\nHere are some 
points to open discussion:\r\n- if you want to prepare your code/datasets on your machine (having internet 
connexion) but run it on another offline machine (not having internet connexion), it won\'t work as is, even if you
have all files locally on this machine.\r\n- AFAIK, you can make it work if you manually put the python files 
(csv.py for example) on this offline machine and change your code to `datasets.load_dataset("MY_PATH/csv.py", 
...)`. But it would be much better if you could run ths same code without modification if files are available 
locally.\r\n- I\'ve also been considering the requirement of downloading Python code and execute on your machine to
use datasets. This can be an issue in a professional context. Downloading a CSV/H5 file is acceptable, downloading 
an executable script can open many security issues. We certainly need a mechanism to at least "freeze" the dataset 
code you retrieved once so that you can review it if you want and then be sure you use this one everywhere and not 
a version dowloaded from internet.\r\n \r\nWDYT? (thks)\r\n\r\n',
    'comment_length': 47,
    'text': 'Discussion using datasets in offline mode \n `datasets.load_dataset("csv", ...)` breaks if you have no
connection (There is already this issue https://github.com/huggingface/datasets/issues/761 about it). It seems to 
be the same for metrics too.\r\n\r\nI create this ticket to discuss a bit and gather what you have in mind or other
propositions.\r\n\r\nHere are some points to open discussion:\r\n- if you want to prepare your code/datasets on 
your machine (having internet connexion) but run it on another offline machine (not having internet connexion), it 
won\'t work as is, even if you have all files locally on this machine.\r\n- AFAIK, you can make it work if you 
manually put the python files (csv.py for example) on this offline machine and change your code to 
`datasets.load_dataset("MY_PATH/csv.py", ...)`. But it would be much better if you could run ths same code without 
modification if files are available locally.\r\n- I\'ve also been considering the requirement of downloading Python
code and execute on your machine to use datasets. This can be an issue in a professional context. Downloading a 
CSV/H5 file is acceptable, downloading an executable script can open many security issues. We certainly need a 
mechanism to at least "freeze" the dataset code you retrieved once so that you can review it if you want and then 
be sure you use this one everywhere and not a version dowloaded from internet.\r\n \r\nWDYT? (thks)\r\n\r\n \n here
is my way to load a dataset offline, but it **requires** an online machine\r\n1. (online machine)\r\n```\r\nimport 
datasets\r\ndata = datasets.load_dataset(...)\r\ndata.save_to_disk(/YOUR/DATASET/DIR)\r\n```\r\n2. copy the dir 
from online to the offline machine\r\n3. (offline machine)\r\n```\r\nimport datasets\r\ndata = 
datasets.load_from_disk(/SAVED/DATA/DIR)\r\n```\r\n\r\nHTH.'
}
score: 0.8566680550575256

{
    'html_url': 'https://github.com/huggingface/datasets/issues/824',
    'title': 'Discussion using datasets in offline mode',
    'comments': "Requiring online connection is a deal breaker in some cases unfortunately so it'd be great if 
offline mode is added similar to how `transformers` loads models offline fine.\r\n\r\n@mandubian's second bullet 
point suggests that there's a workaround allowing you to use your offline (custom?) dataset with `datasets`. Could 
you please elaborate on how that should look like?",
    'body': '`datasets.load_dataset("csv", ...)` breaks if you have no connection (There is already this issue 
https://github.com/huggingface/datasets/issues/761 about it). It seems to be the same for metrics too.\r\n\r\nI 
create this ticket to discuss a bit and gather what you have in mind or other propositions.\r\n\r\nHere are some 
points to open discussion:\r\n- if you want to prepare your code/datasets on your machine (having internet 
connexion) but run it on another offline machine (not having internet connexion), it won\'t work as is, even if you
have all files locally on this machine.\r\n- AFAIK, you can make it work if you manually put the python files 
(csv.py for example) on this offline machine and change your code to `datasets.load_dataset("MY_PATH/csv.py", 
...)`. But it would be much better if you could run ths same code without modification if files are available 
locally.\r\n- I\'ve also been considering the requirement of downloading Python code and execute on your machine to
use datasets. This can be an issue in a professional context. Downloading a CSV/H5 file is acceptable, downloading 
an executable script can open many security issues. We certainly need a mechanism to at least "freeze" the dataset 
code you retrieved once so that you can review it if you want and then be sure you use this one everywhere and not 
a version dowloaded from internet.\r\n \r\nWDYT? (thks)\r\n\r\n',
    'comment_length': 57,
    'text': 'Discussion using datasets in offline mode \n `datasets.load_dataset("csv", ...)` breaks if you have no
connection (There is already this issue https://github.com/huggingface/datasets/issues/761 about it). It seems to 
be the same for metrics too.\r\n\r\nI create this ticket to discuss a bit and gather what you have in mind or other
propositions.\r\n\r\nHere are some points to open discussion:\r\n- if you want to prepare your code/datasets on 
your machine (having internet connexion) but run it on another offline machine (not having internet connexion), it 
won\'t work as is, even if you have all files locally on this machine.\r\n- AFAIK, you can make it work if you 
manually put the python files (csv.py for example) on this offline machine and change your code to 
`datasets.load_dataset("MY_PATH/csv.py", ...)`. But it would be much better if you could run ths same code without 
modification if files are available locally.\r\n- I\'ve also been considering the requirement of downloading Python
code and execute on your machine to use datasets. This can be an issue in a professional context. Downloading a 
CSV/H5 file is acceptable, downloading an executable script can open many security issues. We certainly need a 
mechanism to at least "freeze" the dataset code you retrieved once so that you can review it if you want and then 
be sure you use this one everywhere and not a version dowloaded from internet.\r\n \r\nWDYT? (thks)\r\n\r\n \n 
Requiring online connection is a deal breaker in some cases unfortunately so it\'d be great if offline mode is 
added similar to how `transformers` loads models offline fine.\r\n\r\n@mandubian\'s second bullet point suggests 
that there\'s a workaround allowing you to use your offline (custom?) dataset with `datasets`. Could you please 
elaborate on how that should look like?'
}
score: 0.8541386723518372